In [ ]:
import numpy as np
import os
import torch
import random
from torch.utils.data import DataLoader

# potential mass parametrizations
from differentiable_robot_model.rigid_body_params import (
    UnconstrainedScalar,
    PositiveScalar,
    UnconstrainedTensor,
)

# potential inertia matrix parametrizations
from differentiable_robot_model.rigid_body_params import (
    CovParameterized3DInertiaMatrixNet,
    Symm3DInertiaMatrixNet,
    SymmPosDef3DInertiaMatrixNet,
    TriangParam3DInertiaMatrixNet,
)

from differentiable_robot_model.robot_model import (
    DifferentiableRobotModel,
    DifferentiableKUKAiiwa,
)
from differentiable_robot_model.data_utils import (
    generate_sine_motion_forward_dynamics_data,
)
import diff_robot_data

torch.set_printoptions(precision=3, sci_mode=False)

random.seed(0)
np.random.seed(1)
torch.manual_seed(0)


In [ ]:
class NMSELoss(torch.nn.Module):
    def __init__(self, var):
        super(NMSELoss, self).__init__()
        self.var = var

    def forward(self, yp, yt):
        err = (yp - yt) ** 2
        werr = err / self.var
        return werr.mean()


In [6]:
urdf_path = os.path.join("a1.urdf")
device = "cpu"
learnable_robot_model = DifferentiableRobotModel(
    urdf_path, "A1", device=device
)

learnable_robot_model.make_link_param_learnable(
    "FR_hip", "trans", UnconstrainedTensor(dim1=1, dim2=3)
)
learnable_robot_model.make_link_param_learnable(
        "FR_thigh_shoulder", "trans", UnconstrainedTensor(dim1=1, dim2=3)
    )
learnable_robot_model.make_link_param_learnable(
        "FR_thigh", "trans", UnconstrainedTensor(dim1=1, dim2=3)
    )
learnable_robot_model.make_link_param_learnable(
        "FR_calf", "trans", UnconstrainedTensor(dim1=1, dim2=3)
    )
learnable_robot_model.make_link_param_learnable(
        "FR_foot", "trans", UnconstrainedTensor(dim1=1, dim2=3)
    )
learnable_robot_model.make_link_param_learnable(
        "FL_hip", "trans", UnconstrainedTensor(dim1=1, dim2=3)
    )
learnable_robot_model.make_link_param_learnable(
        "FL_thigh_shoulder", "trans", UnconstrainedTensor(dim1=1, dim2=3)
    )
learnable_robot_model.make_link_param_learnable(
        "FL_thigh", "trans", UnconstrainedTensor(dim1=1, dim2=3)
    )
learnable_robot_model.make_link_param_learnable(
        "FL_calf", "trans", UnconstrainedTensor(dim1=1, dim2=3)
    )
learnable_robot_model.make_link_param_learnable(
        "FL_foot", "trans", UnconstrainedTensor(dim1=1, dim2=3)
    )
learnable_robot_model.make_link_param_learnable(
        "RR_hip", "trans", UnconstrainedTensor(dim1=1, dim2=3)
    )
learnable_robot_model.make_link_param_learnable(
        "RR_thigh_shoulder", "trans", UnconstrainedTensor(dim1=1, dim2=3)
    )
learnable_robot_model.make_link_param_learnable(
        "RR_thigh", "trans", UnconstrainedTensor(dim1=1, dim2=3)
    )
learnable_robot_model.make_link_param_learnable(
        "RR_calf", "trans", UnconstrainedTensor(dim1=1, dim2=3)
    )
learnable_robot_model.make_link_param_learnable(
        "RR_foot", "trans", UnconstrainedTensor(dim1=1, dim2=3)
    )
learnable_robot_model.make_link_param_learnable(
        "RL_hip", "trans", UnconstrainedTensor(dim1=1, dim2=3)
    )
learnable_robot_model.make_link_param_learnable(
        "RL_thigh_shoulder", "trans", UnconstrainedTensor(dim1=1, dim2=3)
    )
learnable_robot_model.make_link_param_learnable(
        "RL_thigh", "trans", UnconstrainedTensor(dim1=1, dim2=3)
    )
learnable_robot_model.make_link_param_learnable(
        "RL_calf", "trans", UnconstrainedTensor(dim1=1, dim2=3)
    )
learnable_robot_model.make_link_param_learnable(
        "RL_foot", "trans", UnconstrainedTensor(dim1=1, dim2=3)
    )

# learnable_robot_model.print_learnable_params()
learnable_robot_model.compute_forward_kinematics(torch.zeros((1,12)),"FR_foot")

Unknown tag "hardwareInterface" in /robot[@name='a1']/transmission[@name='FR_hip_tran']/actuator[@name='FR_hip_motor']
Unknown tag "hardwareInterface" in /robot[@name='a1']/transmission[@name='FR_thigh_tran']/actuator[@name='FR_thigh_motor']
Unknown tag "hardwareInterface" in /robot[@name='a1']/transmission[@name='FR_calf_tran']/actuator[@name='FR_calf_motor']
Unknown tag "hardwareInterface" in /robot[@name='a1']/transmission[@name='FL_hip_tran']/actuator[@name='FL_hip_motor']
Unknown tag "hardwareInterface" in /robot[@name='a1']/transmission[@name='FL_thigh_tran']/actuator[@name='FL_thigh_motor']
Unknown tag "hardwareInterface" in /robot[@name='a1']/transmission[@name='FL_calf_tran']/actuator[@name='FL_calf_motor']
Unknown tag "hardwareInterface" in /robot[@name='a1']/transmission[@name='RR_hip_tran']/actuator[@name='RR_hip_motor']
Unknown tag "hardwareInterface" in /robot[@name='a1']/transmission[@name='RR_thigh_tran']/actuator[@name='RR_thigh_motor']
Unknown tag "hardwareInterface" 

(tensor([[-0.109,  0.257, -0.011]], grad_fn=<AddBackward0>),
 tensor([[0., 0., 0., 1.]], grad_fn=<CopySlices>))

In [12]:
target_ee = torch.rand((1,3))*3

In [18]:
joint_angles = torch.zeros((1,12), requires_grad=True)

In [21]:
optimizer = torch.optim.Adam(learnable_robot_model.parameters(), lr=1e-2)
for _ in range(1000):
    optimizer.zero_grad()
    cur_ee = learnable_robot_model.compute_forward_kinematics(joint_angles,"FR_foot")[0]
    loss = torch.sum(torch.abs(cur_ee - target_ee))
    print(loss)
    loss.backward()
    optimizer.step()
    joint_angles.data -= 0.05*joint_angles.grad.data
    joint_angles.grad.zero_()

tensor(7.179, grad_fn=<SumBackward0>)
tensor(7.092, grad_fn=<SumBackward0>)
tensor(7.006, grad_fn=<SumBackward0>)
tensor(6.919, grad_fn=<SumBackward0>)
tensor(6.833, grad_fn=<SumBackward0>)
tensor(6.746, grad_fn=<SumBackward0>)
tensor(6.659, grad_fn=<SumBackward0>)
tensor(6.572, grad_fn=<SumBackward0>)
tensor(6.485, grad_fn=<SumBackward0>)
tensor(6.398, grad_fn=<SumBackward0>)
tensor(6.310, grad_fn=<SumBackward0>)
tensor(6.223, grad_fn=<SumBackward0>)
tensor(6.135, grad_fn=<SumBackward0>)
tensor(6.047, grad_fn=<SumBackward0>)
tensor(5.959, grad_fn=<SumBackward0>)
tensor(5.870, grad_fn=<SumBackward0>)
tensor(5.782, grad_fn=<SumBackward0>)
tensor(5.693, grad_fn=<SumBackward0>)
tensor(5.604, grad_fn=<SumBackward0>)
tensor(5.515, grad_fn=<SumBackward0>)
tensor(5.425, grad_fn=<SumBackward0>)
tensor(5.336, grad_fn=<SumBackward0>)
tensor(5.246, grad_fn=<SumBackward0>)
tensor(5.156, grad_fn=<SumBackward0>)
tensor(5.066, grad_fn=<SumBackward0>)
tensor(4.976, grad_fn=<SumBackward0>)
tensor(4.885